In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
###### https://www.kaggle.com/c/text-normalization-challenge-english-language


import pandas as pd
import numpy as np
import os
import pickle
import gc
import xgboost as xgb
import numpy as np
import re
import pandas as pd
from sklearn.model_selection import train_test_split

max_num_features = 10
pad_size = 1
boundary_letter = -1
space_letter = 0
max_data_size = 320000

print(os.listdir("../Topic modeling lda/"))

# Any results you write to the current directory are saved as output.

['.ipynb_checkpoints', 'filename.pickle', 'newsgroups', 'Topic_Modeling_LDA.ipynb']


In [4]:
df = pd.read_csv("en_train.csv")

# df = dataframe('../input/en_train.csv')
# # df.head(5)

In [5]:
df.head(15)

,sentence_id,token_id,class,before,after
0,0,0,PLAIN,Brillantaisia,Brillantaisia
1,0,1,PLAIN,is,is
2,0,2,PLAIN,a,a
3,0,3,PLAIN,genus,genus
4,0,4,PLAIN,of,of
5,0,5,PLAIN,plant,plant
6,0,6,PLAIN,in,in
7,0,7,PLAIN,family,family
8,0,8,PLAIN,Acanthaceae,Acanthaceae
9,0,9,PUNCT,.,.


In [17]:
endstring = ''
for i in range(10):
    endstring +="".join(df['before'][i])
endstring

'BrillantaisiaisagenusofplantinfamilyAcanthaceae.'

In [18]:
" ".join(str(df['before'][i]) for item in range(10))

'. . . . . . . . . .'

In [8]:
x_data = []
y_data =  pd.factorize(df['class'])
labels = y_data[1]
y_data = y_data[0]
gc.collect()
for x in df['before'].values:
    x_row = np.ones(max_num_features, dtype=int) * space_letter
    for xi, i in zip(list(str(x)), np.arange(max_num_features)):
        x_row[i] = ord(xi)
    x_data.append(x_row)

In [9]:
def context_window_transform(data, pad_size):
    pre = np.zeros(max_num_features)
    pre = [pre for x in np.arange(pad_size)]
    data = pre + data + pre
    neo_data = []
    for i in np.arange(len(data) - pad_size * 2):
        row = []
        for x in data[i : i + pad_size * 2 + 1]:
            row.append([boundary_letter])
            row.append(x)
        row.append([boundary_letter])
        neo_data.append([int(x) for y in row for x in y])
    return neo_data


In [10]:
x_data = x_data[:max_data_size]
y_data = y_data[:max_data_size]
x_data = np.array(context_window_transform(x_data, pad_size))
gc.collect()
x_data = np.array(x_data)
y_data = np.array(y_data)


In [11]:
print('Total number of samples:', len(x_data))
print('Use: ', max_data_size)
#x_data = np.array(x_data)
#y_data = np.array(y_data)

print('x_data sample:')
print(x_data[0])
print('y_data sample:')
print(y_data[0])
print('labels:')
print(labels)

Total number of samples: 320000
Use:  320000
x_data sample:
[ -1   0   0   0   0   0   0   0   0   0   0  -1  66 114 105 108 108  97
 110 116  97 105  -1 105 115   0   0   0   0   0   0   0   0  -1]
y_data sample:
0
labels:
Index(['PLAIN', 'PUNCT', 'DATE', 'LETTERS', 'CARDINAL', 'VERBATIM', 'DECIMAL',
       'MEASURE', 'MONEY', 'ORDINAL', 'TIME', 'ELECTRONIC', 'DIGIT',
       'FRACTION', 'TELEPHONE', 'ADDRESS'],
      dtype='object')


In [12]:
x_train = x_data
y_train = y_data
gc.collect()

x_train, x_valid, y_train, y_valid= train_test_split(x_train, y_train,
                                                      test_size=0.1, random_state=2017)
gc.collect()
num_class = len(labels)

In [13]:
dtrain = xgb.DMatrix(x_train, label=y_train)
dvalid = xgb.DMatrix(x_valid, label=y_valid)
watchlist = [(dtrain, 'train'),(dvalid, 'valid')]

param = {'objective':'multi:softmax',
         'eta':'0.3', 'max_depth':10,
         'silent':1, 'nthread':-1,
         'num_class':num_class,
         'eval_metric':'merror'}
model = xgb.train(param, dtrain, 50, watchlist, early_stopping_rounds=20,
                  verbose_eval=10)
gc.collect()

[0]	train-merror:0.007781	valid-merror:0.007594
Multiple eval metrics have been passed: 'valid-merror' will be used for early stopping.

Will train until valid-merror hasn't improved in 20 rounds.
[10]	train-merror:0.003542	valid-merror:0.005469
[20]	train-merror:0.002069	valid-merror:0.004875
[30]	train-merror:0.000986	valid-merror:0.004781
[40]	train-merror:0.000503	valid-merror:0.00475
[49]	train-merror:0.000257	valid-merror:0.004781


21

In [14]:
model.save_model('xgb_model')

In [15]:
dvalid = xgb.DMatrix(x_valid)

In [16]:
pred = model.predict(dvalid)
pred = [labels[int(x)] for x in pred]


In [17]:
y_valid = [labels[x] for x in y_valid]
x_valid = [ [ chr(x) for x in y[2 + max_num_features: 2 + max_num_features * 2]] for y in x_valid]
x_valid = [''.join(x) for x in x_valid]
x_valid = [re.sub('a+$', '', x) for x in x_valid]

gc.collect()

0

In [18]:
df_pred = pd.DataFrame(columns=['data', 'predict', 'target'])
df_pred['data'] = x_valid
df_pred['predict'] = pred
df_pred['target'] = y_valid
#df_pred.to_csv(os.path.join(out_path, 'pred.csv'))

# df_erros = df_pred.loc[df_pred['predict'] != df_pred['target']]
# df_erros.to_csv(os.path.join(out_path, 'errors.csv'), index=False)

#model.save_model(os.path.join(out_path, 'xgb_model'))

In [19]:
df_pred[df_pred['predict']=='DATE']

,data,predict,target
28,September,DATE,DATE
53,2011-05-10,DATE,DATE
68,1959      ,DATE,DATE
82,November 1,DATE,DATE
108,"March 1, 2",DATE,DATE
109,26 June   ,DATE,DATE
129,6 January,DATE,DATE
130,1997      ,DATE,DATE
152,1996      ,DATE,DATE
159,18 January,DATE,DATE


In [20]:
test_df = pd.read_csv("en_test.csv")
gc.collect()

11

In [21]:
test_df.head()

,sentence_id,token_id,before
0,0,0,Another
1,0,1,religious
2,0,2,family
3,0,3,is
4,0,4,of


In [22]:
test_data = []
for x in test_df['before'].values:
    x_row = np.ones(max_num_features, dtype=int) * space_letter
    for xi, i in zip(list(str(x)), np.arange(max_num_features)):
        x_row[i] = ord(xi)
    test_data.append(x_row)

In [23]:
#test_data = test_data[:max_data_size]
test_data = np.array(context_window_transform(test_data, pad_size))
gc.collect()
test_data = np.array(test_data)

In [24]:
print('Total number of samples:', len(test_data))
print('Use: ', max_data_size)
#x_data = np.array(x_data)
#y_data = np.array(y_data)

print('x_data sample:')
print(test_data.shape)


Total number of samples: 1088564
Use:  320000
x_data sample:
(1088564, 34)


In [25]:
dtest = xgb.DMatrix(test_data)
t_pred = model.predict(dtest)
t_pred = [labels[int(x)] for x in t_pred]


In [26]:
test_df['predict'] = t_pred

In [27]:
test_df[test_df['predict']=='TELEPHONE']

,sentence_id,token_id,before,predict
7907,513,0,09/08/1943,TELEPHONE
8039,522,1,0-19-511967-3,TELEPHONE
9088,587,11,045007,TELEPHONE
14502,952,1,0-521-04656-4,TELEPHONE
23352,1522,1,978-1-86395-429-7,TELEPHONE
27486,1786,1,0573430,TELEPHONE
28168,1829,22,2001-11-10,TELEPHONE
29659,1926,3,978-0-3743-5095-6,TELEPHONE
32553,2110,1,4-8179-2022,TELEPHONE
32640,2115,1,22-25 RUS,TELEPHONE


In [28]:
test_df2 = pd.read_csv("en_test_2.csv")
gc.collect()
test_df2.head()

,sentence_id,token_id,before
0,0,0,Last
1,0,1,modified
2,0,2,2016-03-31
3,0,3,.
4,1,0,There's


In [29]:
test_data2 = []
for x in test_df2['before'].values:
    x_row = np.ones(max_num_features, dtype=int) * space_letter
    for xi, i in zip(list(str(x)), np.arange(max_num_features)):
        x_row[i] = ord(xi)
    test_data2.append(x_row)

In [30]:
#test_data = test_data[:max_data_size]
test_data2 = np.array(context_window_transform(test_data2, pad_size))
gc.collect()
test_data2 = np.array(test_data2)

In [31]:
print('Total number of samples:', len(test_data2))
print('test_data sample:')
print(test_data2.shape)


Total number of samples: 956046
test_data sample:
(956046, 34)


In [32]:
dtest2 = xgb.DMatrix(test_data2)
t_pred2 = model.predict(dtest2)
t_pred2 = [labels[int(x)] for x in t_pred2]
test_df2['predict'] = t_pred2

In [206]:
text="arrage a reminder for meeting on 2 march 2018 at 7 p.m"

In [214]:
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
token=nltk.word_tokenize(text)
bigrams=ngrams(token,3)

In [215]:
date = []
for a in bigrams:
    date.append(' '.join(map(str,a)))
date

['arrage a reminder',
 'a reminder for',
 'reminder for meeting',
 'for meeting on',
 'meeting on 2',
 'on 2 march',
 '2 march 2018',
 'march 2018 at',
 '2018 at 7',
 'at 7 p.m']

In [216]:
test_data2 = []
for x in date:
    x_row = np.ones(max_num_features, dtype=int) * space_letter
    for xi, i in zip(list(str(x)), np.arange(max_num_features)):
        x_row[i] = ord(xi)
    test_data2.append(x_row)

In [217]:
test_data2 = np.array(context_window_transform(test_data2, pad_size))
gc.collect()
test_data2 = np.array(test_data2)

In [218]:
print('Total number of samples:', len(test_data2))
print('test_data sample:')
print(test_data2.shape)


Total number of samples: 10
test_data sample:
(10, 34)


In [219]:
dtest2 = xgb.DMatrix(test_data2)
t_pred2 = model.predict(dtest2)
t_pred2 = [labels[int(x)] for x in t_pred2]


In [220]:
t_pred2

['PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'DATE',
 'DATE',
 'DATE',
 'ELECTRONIC']